In [1]:
import mmcv

mmcv.collect_env()

from mmcv.runner import load_checkpoint
from mmdet.apis import inference_detector
from mmrotate.models import build_detector

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import rotate

import grpc

import os
import sys
import random
import cv2
import time


class ImageProcessingLayer:
    def __init__(
        self,
        mock=True,
        mock_image_path=None,
        mock_num_samples=10,
        mock_wait_time=1,
    ):
        self.mock = mock

        if not mock:
            return

        self.mock_wait_time = mock_wait_time

        if mock_image_path is None:
            mock_image_path = "data/demo.jpg"

        self._mock_img_full = np.asarray(Image.open(mock_image_path))
        self._output_dim = (200, 200)
        diag_len = np.sqrt(self._output_dim[0] ** 2 + self._output_dim[1] ** 2)
        self._gcps_pixels = self._generate_random_gcps(
            self._mock_img_full, mock_num_samples, padding=(diag_len, diag_len)
        )

        # maybe convert from pixels to lat/lon here

        self._path_pixels = self._build_path_pixels(self._gcps_pixels)

    def _generate_random_gcps(self, img, num_samples, padding=(0, 0)):
        return np.random.randint(
            padding,
            high=(img.shape[0] - padding[0], img.shape[1] - padding[1]),
            size=(num_samples, 2),
        )

    def _build_path_pixels(self, gcps):
        delta = np.diff(gcps, axis=0)
        directions = delta / np.linalg.norm(delta, axis=1).reshape(-1, 1)
        angles = np.arctan2(directions.T[1], directions.T[0]) * 180 / np.pi
        delta_angles = np.append(np.diff(angles), 0)

        path = []

        for t1, t2, angle, delta_angle in zip(gcps, gcps[1:], angles, delta_angles):
            steps = np.linalg.norm(t2 - t1) / 90
            line = np.linspace(t1, t2, steps.astype("uint32"), dtype="uint32")
            path.extend([np.array([x, y, angle]) for x, y in line])

            if delta_angle == 0:
                continue

            if len(line) == 0:
                continue

            interpolated_angles = np.linspace(angle, angle + delta_angle, 3)
            path.extend(
                [
                    np.array([line[-1][0], line[-1][1], theta])
                    for theta in interpolated_angles
                ]
            )

        return path

    def _next_image(self):
        if self.mock_wait_time > 0:
            time.sleep(self.mock_wait_time)

        sample_diag = np.sqrt(self._output_dim[0] ** 2 + self._output_dim[1] ** 2)

        for x, y, theta in self._path_pixels:
            sample = self._crop_around(
                self._mock_img_full, (y, x), (sample_diag, sample_diag)
            )
            rotated_img = self._center_crop(
                rotate(sample, -theta, reshape=False), self._output_dim
            )
            yield rotated_img

    def _crop_around(self, img, center, dim):
        dim = np.array(dim).astype("uint32")
        x = int(center[1] - dim[1] // 2)
        y = int(center[0] - dim[0] // 2)
        return img[y : y + dim[0], x : x + dim[1]]

    def _center_crop(self, img, dim):
        return img[
            img.shape[0] // 2 - dim[0] // 2 : img.shape[0] // 2 + dim[0] // 2,
            img.shape[1] // 2 - dim[1] // 2 : img.shape[1] // 2 + dim[1] // 2,
        ]

    def run(self, img=None):
        if not self.mock:
            assert img is not None, "Image cannot be None"
            return img

        return self._next_image()


class ObjectDetectionLayer:
    def __init__(
        self, config_file=None, checkpoint_file=None, device="cuda", min_confidence=0.3
    ):
        if config_file is None:
            config_file = "examples/oriented_rcnn_r50_fpn_1x_dota_le90.py"
        if checkpoint_file is None:
            checkpoint_file = "examples/oriented_rcnn_r50_fpn_1x_dota_le90-6d2b2ce0.pth"

        self.config_file = config_file
        self.checkpoint_file = checkpoint_file
        self.device = device

        self.model = self._load_model()
        self.min_confidence = min_confidence

    def _load_model(self):
        config = mmcv.Config.fromfile(self.config_file)
        config.model.pretrained = None

        model = build_detector(config.model)
        checkpoint = load_checkpoint(
            model, self.checkpoint_file, map_location=self.device
        )

        model.CLASSES = checkpoint["meta"]["CLASSES"]
        model.cfg = config
        model.to(self.device)
        model = model.eval()

        return model

    def _get_bboxes_pixels(self, img):
        padded_img = np.zeros((max(img.shape[0], 1024), max(img.shape[1], 1024), 3))
        padded_img[: img.shape[0], : img.shape[1]] = img

        vehicle_classes = [
            i for i, c in enumerate(self.model.CLASSES) if "vehicle" in c
        ]

        inference = inference_detector(self.model, padded_img)
        bboxes = [inference[index] for index in vehicle_classes]

        bboxes = np.concatenate(bboxes, axis=0)
        bboxes = bboxes[bboxes[:, 5] > self.min_confidence]

        # the bboxes are in a weird polygonal format, so we convert them to rectangles
        rect_bboxes = (
            np.array(
                [
                    bboxes[:, 1] - bboxes[:, 2] // 2,
                    bboxes[:, 1] + bboxes[:, 2] // 2,
                    bboxes[:, 0] - bboxes[:, 2] // 2,
                    bboxes[:, 0] + bboxes[:, 3],
                    100 * bboxes[:, -1],  # confidence score
                ]
            )
            .astype(int)
            .T
        )

        # follows the format of x0, x1, y0, y1, confidence
        return rect_bboxes

    def run(self, img):
        result = self._get_bboxes_pixels(img)

        # convert pixels to lat/lon here
        return result


class MavlinkInterfaceLayer:
    def __init__(self, protos_path="pipelined_grpc/protos"):
        self.protos_path = protos_path
        
        sys.path.append(os.path.join(os.getcwd(), self.protos_path))
        import messaging_pb2 as messaging_pb2
        import messaging_pb2_grpc as messaging_pb2_grpc

        self.messaging_pb2 = messaging_pb2
        
        self.channel = grpc.insecure_channel("localhost:54051")
        self.stub = messaging_pb2_grpc.MessagingServiceStub(self.channel)

    def run(self, bboxes):
        if len(bboxes) == 0:
            return

        print(bboxes)

        message = self.messaging_pb2.ProcessedDataRequest(request=str(bboxes))
        response = self.stub.RequestProcessedData(message)
        return response

/home/allanlago/anaconda3/envs/rt-flight/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
img_layer = ImageProcessingLayer(mock_wait_time=1)
obj_layer = ObjectDetectionLayer()
mav_layer = MavlinkInterfaceLayer()

for img in img_layer.run():
    bboxes = obj_layer.run(img)
    response = mav_layer.run(bboxes)
    if len(bboxes):
        print(response)

    # for bbox in bboxes:
    #     plt.imshow(img[bbox[0]:bbox[1], bbox[2]:bbox[3]])
    #     plt.title("Confidence: {}".format(bbox[4]))
    #     plt.show()

/home/allanlago/Documents/jupyter-notebooks/rt-flight/examples/mmdetection/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


load checkpoint from local path: examples/oriented_rcnn_r50_fpn_1x_dota_le90-6d2b2ce0.pth


/home/allanlago/Documents/jupyter-notebooks/rt-flight/examples/mmdetection/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '


None
None
None
None
None
None
None
None
None
None
None
None
[[149 163  58  71  79]
 [157 169  57  68  77]
 [172 186  51  62  63]
 [167 181  52  64  53]
 [161 175  54  66  49]
 [179 191  49  61  49]
 [183 197  47  59  45]]

[[149 163  58  71  79]
 [157 169  57  68  77]
 [172 186  51  62  63]
 [167 181  52  64  53]
 [161 175  54  66  49]
 [179 191  49  61  49]
 [183 197  47  59  45]]

[[72 86 29 42 77]
 [71 85 22 35 47]]

[[ 28  40  98 110  85]
 [ 21  33  97 109  77]
 [ 15  27  96 107  64]
 [  8  22  94 106  62]
 [  3  17  93 105  58]
 [ 62  76 103 115  32]
 [ -3  10  92 104  31]]

[[ 28  40  98 110  85]
 [ 21  33  97 109  77]
 [ 15  27  96 107  64]
 [  8  22  94 106  62]
 [  3  17  93 105  58]
 [ 62  76 103 115  32]
 [ -3  10  92 104  31]]

None
None
None
None
None
None
None
None
None
None
None
None
[[  3  15 112 123  60]
 [ 17  31 104 116  60]
 [ 13  25 107 119  59]
 [  6  20 109 121  57]
 [ 24  36 102 113  55]
 [ 28  42 100 112  46]
 [ 46  58  93 104  43]
 [ -4   9 114 126  31]
 [ 50 